<a href="https://colab.research.google.com/github/thehapyone/Thesis_Project/blob/master/Category_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Category Pipeline
This pipeline ensures the categorical features are ready for the embedding stage

In [0]:
import pandas as pd
import numpy as np
import copy as copy
# using interativeimputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## Transform Date Information as Categorical
Here we will transform the day, month and year data as categorical.

In [0]:
# function will transform the day, month and year data as categorical.
def dateTrasfromToCategory(raw_data, debug=False):
  output = copy.copy(raw_data)
  #
  date_columns = ['DAY', 'MONTH', 'YEAR']
  # need to change it to the object dtypes
  date_df = raw_data[date_columns]
  # cast to object
  date_df = date_df.astype('object')
  # updates the original data frame
  output.updates(date_df)
  if debug == True:
    print ('Dtypes are: ')
    print (output[date_columns].dtypes)
    
  return output


## Remove Zero Variance Categories
This removes all feature with no changes to their elements


In [0]:
def removeZeroCategory(raw_data, debug=False):
  # finding column data with almost zero standard deviation. 
  # find all categorical features.
  data_cat = raw_data.select_dtypes(include='object')
  # find categorical features with zero variance - Will don't need those features.
  var_1_categories = []
  for col in data_cat.columns:
    unique_Values = data_cat.loc[:, col].nunique()
    if debug == True:
      print (col, unique_Values)
    # this columns have zero standard deviation
    if unique_Values == 1:
      var_1_categories.append(col)

  # drop all cat features with zero variance
  output = raw_data.drop(labels=var_1_categories, axis=1)
  return output

## Embedding Size Extractor
Here, we will find the cardinality size for each categorical features and its correct embedding size.

In order to apply embeddings for out categorical variables. We need to first determine which features are categorical variable and which are not. We need to ensure we caputure all possible cases of the categorical features.

Okay, then for each categorical variable we need to capture the cardinalty of the feature iteself

In [0]:
# Function takes a data frame containing categorical features alone
# input: category dataframe and the max embedding size
# output: Dataframe (Features, Cardinality, Embedding_Size)
def embeddingSizeExtract(data_categorical, debug=True, max_embedding_size=10):
  # input is categorical features only
  cat_temp = copy.copy(data_categorical)
  
  # We don't need the chassis information here
  try:
    cat_temp = cat_temp.drop(labels=['T_CHASSIS'], axis=1)
  except:
    pass
  # feature names
  categorical_features = data_cat.columns

  # fetch cardinality of the categorical variables.
  cat_cardinality = [(cols, data_cat[cols].nunique()) for cols in categorical_features]

  # check out which feature has the maximum cardinality
  if debug == True:
    max_cad_feature = np.argmax(np.array(cat_cardinality)[:,1].astype(int))
    max_cardinality = cat_cardinality[max_cad_feature]
    print ("Feature with highest cardinality is: ", max_cardinality)

  # Calculate the embedding size for each feature
  # max size of 10
  embedding_size = np.array([(col_names, c, min(max_embedding_size, (c+1)//2)) for col_names, c in cat_cardinality])
  
  # create a dataframe of the features and their sizes
  cat_features_df = pd.DataFrame(data=embedding_size, columns=['Feature', 'Cardinality', 'Embedding_Size'])
  # transform the dtypes
  cat_features_df = cat_features_df.astype({'Cardinality': 'int32', 'Embedding_Size': 'int32'}, copy=False)
  
  # thats the end. We have out cardinality, features and their corresponding embedding size
  return cat_features_df